# Pre-merging baseline correction technique

## Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib as mpl
import pandas as pd        
import math

import sys
!{sys.executable} -m pip install pyFAI
import pyFAI
from pyFAI.gui import jupyter

!{sys.executable} -m pip install pybaselines
from pybaselines import Baseline
from pybaselines.utils import gaussian

pi = math.pi

## Specify diffraction frame to read in

In [ ]:
# data file from I11 in form i11-1-61284.nxs, adjust depending on data name and format


data_file_no = "61284"

data_dir = "C:/path/to/your/data/file/"

data_file = "i11-1-" + data_file_no + ".nxs"

file_path = os.path.join(data_dir, data_file)

In [ ]:
# creation of folder to store processed information
processing_folder = "processed_patterns/individual_frames/"
processing_path = os.path.join(data_dir,processing_folder)


if not os.path.exists(processing_path):
    os.makedirs(processing_path)

In [ ]:
with h5py.File(file_path, 'r') as dat:
    
    #load in as (1, 1441, 1440) shape array
    myData = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
    
    #extract only (1441,1440)
    diff_frame = myData.reshape(myData.shape[1:])
    
total_intensity = np.sum(diff_frame)

diff_frame

In [ ]:
# visualise chosen diffraction frame

plt.rcParams["figure.figsize"]=14,14
plt.title(data_file, fontsize=24)
plt.imshow(diff_frame, cmap='gray', vmin=0, vmax=300)
plt.colorbar()
plt.show()

# Load calibration file

calibrated using pyFAI-calib2 tool

pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
# load the calibration .poni file 
calibration = pyFAI.load('C:/poni/file/location/calibration.poni')
calibration

In [ ]:
#load mask for detector frame
oneD_mask = np.load("C:/mask/file/location/mask.npy")

## Integration using pyFAI module
#### pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
oneD_integrated_plot = calibration.integrate1d(diff_frame, 1000, unit=pyFAI.units.TTH_DEG, radial_range=[0,30], mask=mask.npy)
jupyter.plot1d(oneD_integrated_plot_thresholded)
plt.title(threshold_pattern_name, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 30)
plt.ylabel('Intensity')
#plt.savefig(processing_path + threshold_pattern_name +"_1D_plot")

# Baseline correction 

### using https://pybaselines.readthedocs.io/en/latest/ mor() baseline correction

In [ ]:

intensity_values = oneD_integrated_plot[1]
two_theta = oneD_integrated_plot[0]
x = two_theta
y = intensity_values

baseline_fitter = Baseline(x_data=x)

In [ ]:
len(oneD_integrated_plot)

In [ ]:
# baseline correction, change half_window valuue to best fit dataset

half_window = 7

plt.figure()
plt.plot(x, y, label='data')
plt.plot(x, baseline_fitter.mor(y, half_window=half_window)[0], label=f'half_window={half_window}')
plt.legend()

In [ ]:
# baseline correction
corrected_data = y - baseline_fitter.mor(y, half_window=half_window)[0]

In [ ]:
# plotting of baseline corrected pattern against baseline correction parameters

twoD_pattern_name_baseline_corr = data_file_no +"_baseline_corr_half_window_" + str(half_window)

baseline_corr_name = data_file_no + "half_window_" + str(half_window) + "_baseline"

plt.figure()
plt.plot(x, y, label='data')
plt.plot(x, baseline_fitter.mor(y, half_window=half_window)[0], label=f'half_window={half_window}')
plt.plot(x, corrected_data, label='corrected_baseline')
plt.title(baseline_corr_name, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.ylabel('Intensity')
plt.xlim(4, 20)
plt.legend()
#plt.savefig(processing_path + twoD_pattern_name_baseline_corr)

In [ ]:
# final pattern

plt.figure()
plt.plot(x, corrected_data, label= 'final_pattern')
plt.title(twoD_pattern_name_baseline_corr, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.ylabel('Intensity')
plt.xlim(4, 20)
#plt.savefig(processing_path + twoD_pattern_name_baseline_corr + "_final_pattern")

In [ ]:
# exporting final pattern as .xy data

np.savetxt(processing_path+twoD_pattern_name_baseline_corr+ ".xy", np.c_[x, corrected_data])